In [1]:
# 1155202866
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2 as cv
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import random

c:\Users\86136\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [21]:
# Problem 2
class MyDataset(Dataset):
    def __init__(self, X, y):      
        self.X=torch.tensor(X.values, dtype=torch.float32)
        self.y=torch.tensor(y.values, dtype=torch.float32)
 
    def __len__(self):
        return len(self.y)
   
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def problem_2(df, Xlabel, ylabel, hidden=[3,3,3], test_size=0.3, batch_size=100, learning_rate=0.01, max_epochs=50000):
    # write your logic here, model is the trained ANN model
    model = []
    precision = 0
    recall = 0
    f1score = 0
    random_state = 4320  # default is 4320
    module = []
    
    # YOUR CODE HERE: create the list of modules
    n_features = len(Xlabel)
    module.append(nn.Linear(n_features, hidden[0]))
    module.append(nn.ReLU())
    for i in range(len(hidden)-1):
        module.append(nn.Linear(hidden[i], hidden[i+1]))
        module.append(nn.ReLU())
    
    # the output layer is fixed to 1 neural and sigmoid
    module.append(nn.Linear(hidden[-1],1))
    module.append(nn.Sigmoid())
    # create the nn model based on the list
    model = nn.Sequential(*module)
    
    # split the dataset
    X_train, X_test, y_train, y_test = train_test_split(df[Xlabel], df[ylabel], test_size=test_size, random_state=random_state)
    
    # create the DataLoader for the training set
    mytrain = MyDataset(X_train, y_train)
    train_loader = DataLoader(mytrain, batch_size=batch_size, shuffle=False)

    # use MSE loss function, and SGD optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    
    # training loop
    for epoch in range(max_epochs):
        # you can uncomment the line below, to visualize the slow training process
        # print("Debug: at epoch: ", epoch)
        for data, labels in train_loader:
            # YOUR CODE HERE: training loop
            a = 0 # to prevent empty loop
            optimizer.zero_grad()
            output = model(data)
            loss = loss_function(output, labels)
            loss.backward()
            optimizer.step()

    
    
    # YOUR CODE HERE: follow the training set to create dataloader for testing
    mytest = MyDataset(X_test, y_test)
    test_loader = DataLoader(mytest, batch_size=batch_size, shuffle=False)
    
    # then, calculate the metrics
    y_pred = []
    y_true = []
    for data, labels in test_loader:
        output = model(data)
        predicted_classes = (output > 0.5).float()
        y_pred.extend(predicted_classes.view(-1).numpy())
        y_true.extend(labels.view(-1).numpy())
    
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)
    f1score = metrics.f1_score(y_true, y_pred)

    return model, precision, recall, f1score

In [22]:
# Testing: Problem 2
df = pd.read_csv("creditcard_2023.csv")
Xlabel = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"]
ylabel = ["Class"]
model, p, r, f = problem_2(df, Xlabel, ylabel, hidden=[3,5,5,3], max_epochs=100)
print("model: ", model)
print("precision: ", p)
print("recall: ", r)
print("f1-score: ", f)

model:  Sequential(
  (0): Linear(in_features=29, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=5, bias=True)
  (5): ReLU()
  (6): Linear(in_features=5, out_features=3, bias=True)
  (7): ReLU()
  (8): Linear(in_features=3, out_features=1, bias=True)
  (9): Sigmoid()
)
precision:  0.49938741653916724
recall:  1.0
f1-score:  0.6661219255685572


In [13]:
# Problem 3
def problem_3(image_filename):
    # write your logic here, keypoint and descriptor are BRISK object
    keypoint = 0
    descriptor = 0

    img = cv.imread(image_filename, cv.IMREAD_GRAYSCALE)
    sift = cv.SIFT_create()
    keypoint, descriptor = sift.detectAndCompute(img, None)
    
    return keypoint, descriptor

In [15]:
# Testing: Problem 3
kp, des = problem_3("sample1.jpg")
print(kp)
print(len(kp))
print(des)
print(des.shape)

158
[[  0.  35. 145. ...   0.   0.  24.]
 [  1.  55.  20. ...   4.   0.   4.]
 [ 20.  77.   4. ...   0.  23.  16.]
 ...
 [ 52.   0.   0. ...   0.   0.   9.]
 [  9.   0.   5. ...   0.   0.   0.]
 [ 51.   2.   5. ...   0.   0.   2.]]
(158, 128)


In [19]:
# Problem 4
def problem_4(descriptor_list, k=5):
    # write your logic here, visual_words are the cluster centroids
    visual_words = 0
    random_state = 5726
    
    descriptor_array = np.vstack(descriptor_list)
    kmeans = KMeans(n_clusters=k, random_state=random_state).fit(descriptor_array)
    visual_words = kmeans.cluster_centers_
    
    return visual_words

In [20]:
# Testing: Problem 4
kp, des = problem_3("sample1.jpg")
visual_words = problem_4(des)
print(visual_words)
print(visual_words.shape)

[[ 15.133332     6.2000027    4.9666653   17.53333     37.66667
   40.06666     21.100002    19.633331    19.299995    12.933346
   10.466669    20.433332    48.800007    44.66666     26.299995
   24.133333    55.933334    19.466671    13.9000025    8.166664
   10.299999    12.4         37.13333     71.26666     39.1
    9.4333315   11.400001     8.066667     7.7333336   18.333336
   32.          60.43333     14.099998     8.4333315   13.833329
   42.86667     48.433334    26.3         17.766666    21.46667
   36.36667     17.233332    15.166666    56.033333    79.93333
   32.966663    10.066664     9.833332   128.63335     32.1
    7.0333357    7.6000004   10.800001     8.900002    17.733335
   62.766678    51.766666     9.466666     8.833333    13.266666
   15.833336    36.266674    34.233337    44.73333     14.333334
   26.83334     27.133331    51.1         37.8         20.733334
   11.499996     8.566663    43.866665    13.133333    14.366667
   40.76667     73.566666    41.83333 

In [47]:
# Problem 5
# Takes 2 parameters. The first one is a dictionary that holds the descriptors that are separated class by class 
# And the second parameter is an array that holds the central points (visual words) of the k means clustering
# Returns a dictionary that holds the histograms for each images that are separated class by class. 
def image_class(all_bovw, centers):
    dict_feature = {}
    for key,value in all_bovw.items():
        category = []
        for img in value:
            histogram = np.zeros(len(centers))
            for each_feature in img:
                ind = find_index(each_feature, centers)
                histogram[ind] += 1
            category.append(histogram)
        dict_feature[key] = category
    return dict_feature

def find_index(each_feature, centers):
    best = 0
    dist = np.linalg.norm(each_feature-centers[0])
    for k in range(len(centers)):
        if np.linalg.norm(each_feature-centers[k]) < dist:
            best = k
            dist = np.linalg.norm(each_feature-centers[k])
    return best

def problem_5(list_of_image, k=5):
    # write your logic here, visual_bag_of_words is a dictionary
    sift_vectors = {}
    descriptor_list = []
    visual_bag_of_words = {}

    # YOUR CODE HERE:
    # reuse problem 3 and collect the list of all descriptors
    for img in list_of_image:
        kp, des = problem_3(img)
        descriptor_list.append(des)
        sift_vectors[img] = des

    # YOUR CODE HERE:
    # reuse problem 4 and obtain the visual words
    visual_words = problem_4(descriptor_list, k)

    # YOUR CODE HERE:
    # based on the visual words and cluster centers
    # construct the visual bow (dict)
    for img in list_of_image:
        visual_bag_of_words[img] = image_class({img: [sift_vectors[img]]}, visual_words)[img]

    return visual_bag_of_words
    

In [48]:
# Testing: Problem 5
list_of_image = ["sample1.jpg", "sample2.jpg", "sample3.jpg", "sample4.jpg", "sample5.jpg"]
vbow = problem_5(list_of_image, 10)
print(vbow)  

{'sample1.jpg': [array([26.,  5., 31., 18.,  9., 24.,  5.,  5., 33.,  2.])], 'sample2.jpg': [array([ 10.,  27., 130.,  22.,  55.,  36.,  20.,  44.,  55.,  45.])], 'sample3.jpg': [array([37., 15., 47., 39., 75., 61., 42., 31., 64., 25.])], 'sample4.jpg': [array([11., 20., 27., 11.,  7., 63.,  4., 13., 43.,  7.])], 'sample5.jpg': [array([176., 245., 185., 256., 311.,  91., 321., 353., 105., 205.])]}


In [6]:
# Problem 6
from sklearn.model_selection import KFold
def problem_6(df_X, df_y, kfold=10, alpha=1.0):
    # write your logic here, model is the trained ridge model
    model = 0
    rmse = 0
    random_state = 4320  # default is 4320
    random.seed(random_state) # may be useful

    model = Ridge(alpha=alpha)
    kf = KFold(n_splits=kfold, random_state=random_state, shuffle=True)
    
    squared_errors = []
    scaler = StandardScaler()

    for train_index, test_index in kf.split(df_X):
        X_train, X_test = df_X.iloc[train_index], df_X.iloc[test_index]
        y_train, y_test = df_y.iloc[train_index], df_y.iloc[test_index]
        
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        squared_errors.append(metrics.mean_squared_error(y_test, y_pred))

    rmse = np.sqrt(np.mean(squared_errors))

    model.fit(df_X, df_y)
    
    
    return model, rmse

In [7]:
# Testing: Problem 6
df = pd.read_csv('USA_Housing.csv')
df_X = df[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Avg. Area Number of Bedrooms', 'Area Population']]
df_X = pd.DataFrame(StandardScaler().fit_transform(df_X.values))
df_y = df['Price']
model, rmse = problem_6(df_X, df_y, kfold=10)
print("model: ", model)
print("cv rmse: ", rmse)

model:  Ridge()
cv rmse:  101230.77274562986
